In [1]:
from tic_tac_toe import tic_tac_toe as t
import numpy as np
import random
class node:
    def __init__(self,state,player,depth=0,parent=None):
        self.value=0
        
        self.child=list()
        self.player=player 
        self.state=state
        self.parent=parent
        self.depth=depth
        
    def expand(self):
        if self.check_terminal():      
            return
        valid_moves=t.valid_moves(self.state)
        children=list()
        for move in valid_moves:
            child_state=t.state_modify(self.state.copy(),move,self.player)
            children.append(node(child_state,-(self.player),self.depth+1,self))
        self.child=children
        
    def check_terminal(self):
        for row in range(3):
            if np.all(self.state[row, :] == -(self.player)):
                self.value = -self.player/self.depth
                return True
                    
        for col in range(3):
            if np.all(self.state[:, col] == -(self.player)):
                self.value = -self.player/self.depth
                return True
    
        if np.trace(self.state) == -(self.player * 3) or np.trace(np.fliplr(self.state)) == -(self.player * 3):
            self.value = -self.player/self.depth
            return True
    
        if 0 not in self.state.reshape(-1):
            return True
        return False
    
    def __repr__(self):
        return f"{self.state}"          
            
    def sigma_move_max(self):
        maxvalue=list()
        for children in self.child:
            maxvalue.append(children.value)
        if all(i == maxvalue[0] for i in maxvalue):
            return self.child[random.randrange(len(maxvalue))]
        index_max=maxvalue.index(max(maxvalue))
        return self.child[index_max]

    def sigma_move_min(self):
        minvalue=list()
        for children in self.child:
            minvalue.append(children.value)
        if all(i == minvalue[0] for i in minvalue):
            return self.child[random.randrange(len(minvalue))]
        index_min=minvalue.index(min(minvalue))
        return self.child[index_min]
        
    def noob_move(self,action):
        row=action//3
        col=action%3
        self.state[row,col]=self.player
        for children in self.child:
            if (children.state == self.state).all():
                print(children)
                return children
        
    def calculate_win(self):
        child_value=list()
        for children in self.child:
            child_value.append(children.value)
        child_value.sort()
        if self.player==1:
            self.value=child_value.pop(-1)   
            if child_value!=[]:
                if child_value[-1]==self.value:
                    self.value+=child_value.pop(-1)
        if self.player==-1:
            self.value=child_value.pop(0)
            if child_value!=[]:
                if child_value[0]==self.value:
                    self.value+=child_value.pop(0)

    def get_layers_deepest_first(root):
        current_layer=[root]
        next_layer=[]
        deepest_first_list=list()
        
        while True:
            for childrens in current_layer:
                childrens.expand()
                next_layer.extend(childrens.child)
            deepest_first_list=next_layer+deepest_first_list
            current_layer=next_layer.copy()
            next_layer.clear()
            if current_layer==[]:
                break
        return deepest_first_list
        
    def run_backprop(root):
        root.depth=0
        list=node.get_layers_deepest_first(root)
        for nodes in list:
            if nodes.check_terminal():
                continue
            nodes.calculate_win()

Training and Initialization

In [3]:
game=t()
state=game.initialise_state()
human=int(input("As what you wish to play:"))
state=node(state,human)
node.run_backprop(state)

As what you wish to play: 1


List all Child elements with their values

In [24]:
for a in state.child:
    print(a.value,"\n",a.state)

0 
 [[-1.  1.  1.]
 [ 1. -1. -1.]
 [-1.  1.  1.]]


AI playes the best move for the player

In [23]:
state=state.sigma_move_max() if state.player==1 else state.sigma_move_min()  
state.run_backprop
state

[[-1.  1.  1.]
 [ 1. -1. -1.]
 [-1.  0.  1.]]

Player move

In [11]:
move=int(input("Enter a move:"))
state=state.noob_move(move)
state.run_backprop()

Enter a move: 4


[[ 0.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0.  1.]]
